In [1]:
import pandas as pd


In [61]:
df = pd.read_csv("/Users/pedrosanhueza/EXOXY/Administrative Documents/Jobs/geoaire/Planilla I-tree Candelaria.csv")

In [63]:
df.columns = map(str.lower, df.columns)

In [67]:
# Check if 'fecha' is in the columns
if 'fecha' in df.columns:
    # Drop the 'fecha' column
    df = df.drop('fecha', axis=1)

# change column name 'velocidad'
substrings_to_replace_velocidad = ["vv", "viento", "velocidad", "velocidad del viento"]
for column in df.columns:
    if any(substring in column.lower() for substring in substrings_to_replace_velocidad):
        new_column_name = "Velocidad del viento (m/s)"
        df.rename(columns={column: new_column_name}, inplace=True)

# change column name 'mp2,5'
substrings_to_replace_mp = ["mp", "2,5", "25"]
for column in df.columns:
    if any(substring in column.lower() for substring in substrings_to_replace_mp):
        new_column_name = "MP2,5 (µg/m³)"
        df.rename(columns={column: new_column_name}, inplace=True)

# change column name 'altura'
substrings_to_replace_mp = ["mh", "altura", "mezcla"]
for column in df.columns:
    if any(substring in column.lower() for substring in substrings_to_replace_mp):
        new_column_name = "Altura de Mezcla (m)"
        df.rename(columns={column: new_column_name}, inplace=True)

df

,"MP2,5 (µg/m³)",Velocidad del viento (m/s),Altura de Mezcla (m)
0,25.6,"3,0",124.207
1,7.0,"2,6",78.278
2,5.4,"1,8",51.695
3,4.0,"1,4",50.000
4,11.5,"1,2",50.000
...,...,...,...
8317,10.1,"3,7",140.479
8318,11.8,"3,2",64.357
8319,12.5,"3,1",77.685
8320,10.8,"2,4",77.483


In [299]:
B = [4.5,3.0,1.5,1.5,1.5,0.0,1.5,3.0,4.5,6.0]
C = [0.0000007,0.0000003,0.0000001,0.0000004,0.0000004,0.0000000,0.0000005,0.0000013,0.0000024,0.0000017]

F_0 = 0
I_0 = 0

for row_b, row_c in zip(B,C):
    I_1 = ( F_0 + row_c ) * row_b / 100
    print(f"I_1 = {I_1:.20f}".format(I_1))
    print(f"{I_1:.20f} = ( {F_0:.20f} + {row_c:.20f} ) * {row_b:.20f} / {100}")
    F_0 = F_0 + row_c - I_1
    print(f"F_0 = {F_0:.20f}\n".format(F_0))

print(f'''
I_1 = {I_1:.20f}
F_0 = {F_0:.20f}
''')

I_1 = 0.00000003150000000000
0.00000003150000000000 = ( 0.00000000000000000000 + 0.00000070000000000000 ) * 4.50000000000000000000 / 100
F_0 = 0.00000066850000000000

I_1 = 0.00000002905500000000
0.00000002905500000000 = ( 0.00000066850000000000 + 0.00000030000000000000 ) * 3.00000000000000000000 / 100
F_0 = 0.00000093944500000000

I_1 = 0.00000001559167500000
0.00000001559167500000 = ( 0.00000093944500000000 + 0.00000010000000000000 ) * 1.50000000000000000000 / 100
F_0 = 0.00000102385332500000

I_1 = 0.00000002135779987500
0.00000002135779987500 = ( 0.00000102385332500000 + 0.00000040000000000000 ) * 1.50000000000000000000 / 100
F_0 = 0.00000140249552512500

I_1 = 0.00000002703743287687
0.00000002703743287687 = ( 0.00000140249552512500 + 0.00000040000000000000 ) * 1.50000000000000000000 / 100
F_0 = 0.00000177545809224812

I_1 = 0.00000000000000000000
0.00000000000000000000 = ( 0.00000177545809224812 + 0.00000000000000000000 ) * 0.00000000000000000000 / 100
F_0 = 0.00000177545809224812

In [303]:
print(0.1 + 0.2)

0.30000000000000004


In [304]:
"0.30000000000000004"

'0.30000000000000004'